In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech

In [16]:
from malaya_speech.train.model.vits import model
from malaya_speech.train.model import vits

In [5]:
spec_channels = 1024 // 2 + 1
segment_size = 8192 // 256
spec_channels, segment_size

(513, 32)

In [6]:
inter_channels = 192
hidden_channels = 192
filter_channels = 768
gin_channels = 0
encoder = model.TextEncoder(60, inter_channels, hidden_channels, filter_channels, 2, 6, 3, 0.1)
enc_q = model.PosteriorEncoder(spec_channels, inter_channels, hidden_channels, 5, 1, 16, gin_channels=gin_channels)
flow = model.ResidualCouplingBlock(inter_channels, hidden_channels, 5, 1, 4, gin_channels=gin_channels)

In [7]:
x = tf.constant([[1,2,3,4,5], [2,3,0,0,0]])
x_lengths = tf.constant([5, 2])

In [8]:
y = tf.random.normal(shape = (2, 100, spec_channels))
y_lengths = tf.constant([50, 100])

In [9]:
x, m, logs, x_mask = encoder(x, x_lengths, training = True)
x.shape, m.shape, logs.shape

(TensorShape([2, 5, 192]), TensorShape([2, 5, 192]), TensorShape([2, 5, 192]))

In [10]:
g = None

In [11]:
z, m_q, logs_q, y_mask = enc_q(y, y_lengths, g=g)
z_p = flow(z, y_mask, g=g)

In [12]:
z.shape, m_q.shape, logs_q.shape

(TensorShape([2, 100, 192]),
 TensorShape([2, 100, 192]),
 TensorShape([2, 100, 192]))

In [13]:
z_p.shape

TensorShape([2, 100, 192])

In [15]:
malaya_speech.config.vits_base_config

{'model': {'inter_channels': 192,
  'hidden_channels': 192,
  'filter_channels': 768,
  'n_heads': 2,
  'n_layers': 6,
  'kernel_size': 3,
  'p_dropout': 0.1,
  'resblock': '1',
  'resblock_kernel_sizes': [3, 7, 11],
  'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
  'upsample_rates': [8, 8, 2, 2],
  'upsample_initial_channel': 512,
  'upsample_kernel_sizes': [16, 16, 4, 4],
  'n_layers_q': 3,
  'use_spectral_norm': False},
 'train': {'log_interval': 200,
  'eval_interval': 1000,
  'seed': 1234,
  'epochs': 20000,
  'learning_rate': 0.0002,
  'betas': [0.8, 0.99],
  'eps': 1e-09,
  'batch_size': 64,
  'fp16_run': True,
  'lr_decay': 0.999875,
  'segment_size': 8192,
  'init_lr_ratio': 1,
  'warmup_epochs': 0,
  'c_mel': 45,
  'c_kl': 1.0},
 'data': {'max_wav_value': 32768.0,
  'sampling_rate': 22050,
  'filter_length': 1024,
  'hop_length': 256,
  'win_length': 1024,
  'n_mel_channels': 80,
  'mel_fmin': 0.0,
  'mel_fmax': None,
  'add_blank': True,
  'n_speakers': 0}}

In [28]:
vits.HParams(**malaya_speech.config.vits_base_config)

{'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0.1, 'resblock': '1', 'resblock_kernel_sizes': [3, 7, 11], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_rates': [8, 8, 2, 2], 'upsample_initial_channel': 512, 'upsample_kernel_sizes': [16, 16, 4, 4], 'n_layers_q': 3, 'use_spectral_norm': False}, 'train': {'log_interval': 200, 'eval_interval': 1000, 'seed': 1234, 'epochs': 20000, 'learning_rate': 0.0002, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 64, 'fp16_run': True, 'lr_decay': 0.999875, 'segment_size': 8192, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0}, 'data': {'max_wav_value': 32768.0, 'sampling_rate': 22050, 'filter_length': 1024, 'hop_length': 256, 'win_length': 1024, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': None, 'add_blank': True, 'n_speakers': 0}}

In [27]:
HParams(**malaya_speech.config.vits_base_config)

{'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0.1, 'resblock': '1', 'resblock_kernel_sizes': [3, 7, 11], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_rates': [8, 8, 2, 2], 'upsample_initial_channel': 512, 'upsample_kernel_sizes': [16, 16, 4, 4], 'n_layers_q': 3, 'use_spectral_norm': False}, 'train': {'log_interval': 200, 'eval_interval': 1000, 'seed': 1234, 'epochs': 20000, 'learning_rate': 0.0002, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 64, 'fp16_run': True, 'lr_decay': 0.999875, 'segment_size': 8192, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0}, 'data': {'max_wav_value': 32768.0, 'sampling_rate': 22050, 'filter_length': 1024, 'hop_length': 256, 'win_length': 1024, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': None, 'add_blank': True, 'n_speakers': 0}}